1. Попробуйте изменить параметры нейронной сети, работающей с датасетом imdb, либо нейронной сети, работающей airline-passengers (она прилагается вместе с датасетом к уроку в виде отдельного скрипта) так, чтобы улучшить её точность. Приложите анализ

In [38]:
import numpy as np
from keras.utils.data_utils import pad_sequences 
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM
from keras.datasets import imdb

In [4]:
max_features = 10_000

# обрезаем тексты после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 80
batch_size = 50 # увеличьте значение для ускорения обучения

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

In [5]:
[len(i) for i in X_train[:5]]

[218, 189, 141, 550, 147]

In [7]:
print(X_train[:5])

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 

In [8]:
index2word = {v:k for k, v in imdb.get_word_index().items()}

for idx_word in X_train[0]:
    print(index2word[idx_word], end=' ')

1641221/1641221 [==============================] - 0s 0us/step
the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while histor

In [13]:
print('Pad последовательности (примеров в x единицу времени)')
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad последовательности (примеров в x единицу времени)
X_train shape: (25000, 80)
X_test shape: (25000, 80)


In [14]:
X_train[0]

array([  15,  256,    4,    2,    7, 3766,    5,  723,   36,   71,   43,
        530,  476,   26,  400,  317,   46,    7,    4,    2, 1029,   13,
        104,   88,    4,  381,   15,  297,   98,   32, 2071,   56,   26,
        141,    6,  194, 7486,   18,    4,  226,   22,   21,  134,  476,
         26,  480,    5,  144,   30, 5535,   18,   51,   36,   28,  224,
         92,   25,  104,    4,  226,   65,   16,   38, 1334,   88,   12,
         16,  283,    5,   16, 4472,  113,  103,   32,   15,   16, 5345,
         19,  178,   32], dtype=int32)

In [15]:
X_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    1,  591,  202,   14,   31,    6,  717,   10,   10,    2,
          2,    5,    4,  360,    7,    4,  177, 5760,  394,  354,    4,
        123,    9, 1035, 1035, 1035,   10,   10,   13,   92,  124,   89,
        488, 7944,  100,   28, 1668,   14,   31,   23,   27, 7479,   29,
        220,  468,    8,  124,   14,  286,  170,    8,  157,   46,    5,
         27,  239,   16,  179,    2,   38,   32,   25, 7944,  451,  202,
         14,    6,  717], dtype=int32)

In [16]:
index = imdb.get_word_index()
len(index)

88584

In [17]:
np.max(X_train)

9999

# Bag of Word

In [18]:
def vectorize(sequences, dimension=10_000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results
 
X_train_bow = vectorize(X_train, dimension=max_features)
X_test_bow = vectorize(X_test, dimension=max_features)

In [19]:
X_train_bow.shape

(25000, 10000)

In [20]:
X_train_bow[0:, :10]

array([[0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.]])

Построим модель

In [21]:
model_bow = Sequential()

model_bow.add(Dense(128, activation='relu', input_shape=(max_features, )))
model_bow.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимизаторы и другие конфигурации
model_bow.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model_bow.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1280128   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,257
Trainable params: 1,280,257
Non-trainable params: 0
_________________________________________________________________


Обучение модели

In [22]:
model_bow.fit(X_train_bow[:], y_train[:],
              batch_size=batch_size,
              epochs=3,
              validation_data=(X_test_bow, y_test))

Epoch 1/3
500/500 [==============================] - 13s 24ms/step - loss: 0.4013 - accuracy: 0.8176 - val_loss: 0.3662 - val_accuracy: 0.8364
Epoch 2/3
500/500 [==============================] - 11s 21ms/step - loss: 0.2577 - accuracy: 0.8937 - val_loss: 0.3877 - val_accuracy: 0.8275
Epoch 3/3
500/500 [==============================] - 12s 24ms/step - loss: 0.1731 - accuracy: 0.9327 - val_loss: 0.4846 - val_accuracy: 0.8113


Валидация модели

In [23]:
score, acc = model_bow.evaluate(X_test_bow, y_test, batch_size=batch_size)

500/500 [==============================] - 2s 5ms/step - loss: 0.4846 - accuracy: 0.8113


Обучение на последовательности

In [24]:
def vectorize_seq(sequences, xLen=100):
    results = np.zeros((len(sequences), xLen, 1))
    for i, sequence in enumerate(sequences):
        for j in range(xLen):
            results[i, j,:] = sequence[j]
    return results
 
X_train_seq = vectorize_seq(X_train, xLen=maxlen)
X_test_seq = vectorize_seq(X_test, xLen=maxlen)

In [25]:
X_test.shape, X_test_seq.shape

((25000, 80), (25000, 80, 1))

In [26]:
display(X_train[0, :10], X_train_seq[0, :10])

array([  15,  256,    4,    2,    7, 3766,    5,  723,   36,   71],
      dtype=int32)

array([[1.500e+01],
       [2.560e+02],
       [4.000e+00],
       [2.000e+00],
       [7.000e+00],
       [3.766e+03],
       [5.000e+00],
       [7.230e+02],
       [3.600e+01],
       [7.100e+01]])

Построение модели

In [27]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, 1)))
model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимизаторы и другие конфигурации
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


Процесс обучения

In [28]:
model.fit(X_train_seq, y_train,
          batch_size=1000,
          epochs=5,
          validation_data=(X_test_seq, y_test))

Epoch 1/5
25/25 [==============================] - 38s 1s/step - loss: 0.6927 - accuracy: 0.5186 - val_loss: 0.6905 - val_accuracy: 0.5294
Epoch 2/5
25/25 [==============================] - 43s 2s/step - loss: 0.6881 - accuracy: 0.5393 - val_loss: 0.6865 - val_accuracy: 0.5454
Epoch 3/5
25/25 [==============================] - 34s 1s/step - loss: 0.6862 - accuracy: 0.5500 - val_loss: 0.6895 - val_accuracy: 0.5390
Epoch 4/5
25/25 [==============================] - 34s 1s/step - loss: 0.6852 - accuracy: 0.5524 - val_loss: 0.6854 - val_accuracy: 0.5547
Epoch 5/5
25/25 [==============================] - 45s 2s/step - loss: 0.6831 - accuracy: 0.5598 - val_loss: 0.6867 - val_accuracy: 0.5441


Валидация модели

In [29]:
score, acc = model.evaluate(X_test_seq, y_test, batch_size=batch_size)

500/500 [==============================] - 23s 45ms/step - loss: 0.6867 - accuracy: 0.5441


# Embedding
Сеть обучилась не очень хорошо, попробуем использовать эмбединги (векторное представление слов)

In [30]:
model = Sequential()
model.add(Embedding(max_features, 128))

In [31]:
model.predict(X_train[:1])

1/1 [==============================] - 0s 81ms/step


array([[[ 0.00929098,  0.02772431,  0.01669017, ...,  0.04224013,
         -0.02084659, -0.04560492],
        [ 0.04370891, -0.03611586,  0.0200032 , ..., -0.01383482,
          0.03734556, -0.04876429],
        [ 0.04863637,  0.04241306,  0.01804303, ...,  0.02465114,
          0.01699336, -0.04688082],
        ...,
        [ 0.04331944, -0.03064115, -0.01109391, ..., -0.02334991,
          0.04136768, -0.04549614],
        [-0.03899385,  0.04023511,  0.00353663, ..., -0.01442599,
          0.03262058,  0.02145861],
        [-0.00241605,  0.04753423,  0.01982892, ...,  0.00799163,
         -0.04091639,  0.01465843]]], dtype=float32)

In [32]:
model.predict(X_train[:1]).shape

1/1 [==============================] - 0s 20ms/step


(1, 80, 128)

Построение модели

In [34]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Dropout(0.3))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         1280000   
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


Обучение модели

In [35]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=3,
          validation_data=(X_test, y_test))

Epoch 1/3
196/196 [==============================] - 92s 453ms/step - loss: 0.4686 - accuracy: 0.7646 - val_loss: 0.3522 - val_accuracy: 0.8457
Epoch 2/3
196/196 [==============================] - 85s 436ms/step - loss: 0.2983 - accuracy: 0.8772 - val_loss: 0.3521 - val_accuracy: 0.8446
Epoch 3/3
196/196 [==============================] - 83s 424ms/step - loss: 0.2412 - accuracy: 0.9048 - val_loss: 0.3722 - val_accuracy: 0.8382


Валидация модели

In [36]:
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)

500/500 [==============================] - 29s 58ms/step - loss: 0.3722 - accuracy: 0.8382
